In [2]:
import config
import cx_Oracle
import re
import pandas as pd
import csv

pattern = re.compile(r'\s+')

In [65]:
# connect to Oracle DB
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding = "UTF-8"
    )
    print("Connected")
    print(connection.version)
except cx_Oracle.Error as error:
    print(error)

cursor = connection.cursor()

Connected
12.2.0.1.0


In [123]:
# set up style view and get the geometrytype of that style's feature
view_name = "V_ADM_GMIX_LN_1"
# if the logic is more difficult then what the program can handle (mainly having "or"). dont sort logic statement
logichard = False
feature_name = view_name[2:]
print(feature_name)
sql = "Select g3e_geometrytype from G3E_COMPONENT where G3E_NAME = '{name}'".format(name = feature_name)
cursor.execute(sql)
geometrytype = cursor.fetchall()[0][0]
print(geometrytype)

ADM_GMIX_LN_1
PolylineGeometry


In [124]:
# hardcode condition to know which styletable for the given style view
if geometrytype == "PolygonGeometry":
    styletable = 'G3E_AREASTYLE'
elif geometrytype == "PolylineGeometry":
    styletable = 'G3E_COMPOSITELINESTYLE'
elif geometrytype == "OrientedPointGeometry":
    styletable = 'G3E_POINTSTYLE'
elif geometrytype == "TextPointGeometry":
    styletable = 'G3E_TEXTSTYLE'
elif geometrytype == "CompositePolylineGeometry":
    styletable = 'G3E_COMPOSITELINESTYLE'

print(styletable)

G3E_COMPOSITELINESTYLE


In [125]:
# extract text that create the given style view
sql = "SELECT TEXT FROM USER_VIEWS WHERE VIEW_NAME = '{view_name}'".format(view_name = view_name)
cursor.execute(sql)
fetch = cursor.fetchall()
txts = fetch[0][0]
txts = txts.rpartition("CASE")[2]
txts = txts.rpartition("ELSE")[0]
txts = re.sub(pattern,'',txts)
txts = txts.split("WHEN")

In [126]:
# extract cause(logic condition) and effect(styleid) into dict
logics = {}

spl_chr = "THEN"
spl_and = "AND"
spl_or = "OR"
spl_eq = "="
spl_neq = "<>"
spl_geq = ">="
spl_seq = "<="
spl_g = ">"
spl_s = "<"
spl_in = "IN"
spl_notin = "NOTIN"
for txt in txts:
    if txt != "":
        styleid = txt.rpartition(spl_chr)[2]
        content = txt.rpartition(spl_chr)[0]
        if not logichard:
            logic = {}
            if spl_and in content:
                cond_list = content.split(spl_and)
                for cond in cond_list:
                    if spl_eq in cond:
                        logic.update({cond.rpartition(spl_eq)[0]:"= " + cond.rpartition(spl_eq)[2]})
                    elif spl_neq in cond:
                        logic.update({cond.rpartition(spl_neq)[0]:"<> " + cond.rpartition(spl_neq)[2]})
                    elif spl_geq in cond:
                        logic.update({cond.rpartition(spl_geq)[0]:">= " + cond.rpartition(spl_geq)[2]})
                    elif spl.seq in cond:
                        logic.update({cond.rpartition(spl_seq)[0]:"<= " + cond.rpartition(spl_seq)[2]})
                    elif sql.g in cond:
                        logic.update({cond.rpartition(spl_g)[0]:"> " + cond.rpartition(spl_g)[2]})
                    elif sql.s in cond:
                        logic.update({cond.rpartition(spl_s)[0]:"< " + cond.rpartition(spl_s)[2]})
                    elif spl_notin in cond:
                        logic.update({cond.rpartition(spl_notin)[0]:"NOT " + cond.rpartition(spl_notin)[2]})
                    elif spl_in in cond:
                        logic.update({cond.rpartition(spl_in)[0]:"= " + cond.rpartition(spl_in)[2]})
                    
            else:
                if spl_eq in content:
                    logic.update({content.rpartition(spl_eq)[0]:"= " + content.rpartition(spl_eq)[2]})
                elif spl_neq in content:
                    logic.update({content.rpartition(spl_neq)[0]:"<> " + content.rpartition(spl_neq)[2]})
                elif spl_geq in content:
                    logic.update({content.rpartition(spl_geq)[0]:">= " + content.rpartition(spl_geq)[2]})
                elif spl.seq in content:
                    logic.update({content.rpartition(spl_seq)[0]:"<= " + content.rpartition(spl_seq)[2]})
                elif sql.g in content:
                    logic.update({content.rpartition(spl_g)[0]:"> " + content.rpartition(spl_g)[2]})
                elif sql.s in content:
                    logic.update({content.rpartition(spl_s)[0]:"< " + content.rpartition(spl_s)[2]})
                elif spl_notin in content:
                    logic.update({content.rpartition(spl_notin)[0]:"NOT " + content.rpartition(spl_notin)[2]})
                elif spl_in in content:
                    logic.update({content.rpartition(spl_in)[0]:"= " + content.rpartition(spl_in)[2]})
        else:
            logic = content

        index = len(styleid)
        # remove comment that often tag behind style id
        for i in styleid:
            if not i.isdigit():
                index = styleid.index(i)
                break
        styleid = styleid[:index]
        # update logic with corresponding styleid
        logics.update({styleid : logic})
    
print(logics)

{'21220150': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'HVSCH'", "TRIM(D.CODETEXT)IN('BASE','": "= S')"}, '21220160': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'HVMAP11KV'", "TRIM(D.CODETEXT)IN('BASE','": "= S')"}, '21220170': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'HVMAP22KV'", "TRIM(D.CODETEXT)IN('BASE','": "= S')"}, '21220180': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'LVSCH'", 'TRIM(D.CODETEXT)': "= 'BASE'"}, '21220190': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'LVMAP'", 'TRIM(D.CODETEXT)': "= 'BASE'"}, '21220200': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'SOLKOR'", 'TRIM(D.CODETEXT)': "= 'BASE'"}, '21220210': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= ('ADM','DT')", 'TRIM(D.CODETEXT)': "= 'BASE'"}, '21220220': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'PT'", 'TRIM(D.CODETEXT)': "= 'BASE'"}, '21220230': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT)': "= 'TRMAP66KV'", 'TRIM(D.CODETEXT)': "= 'BASE'"}, '21220240': {'A.LINESTYLE': '= 2', 'TRIM(E.CODETEXT

In [127]:
# get the relevant information for the given style id
styles = {}
# check if compositeline, as they have a seperate method of getting style info
if styletable != 'G3E_COMPOSITELINESTYLE':
    for styleid in logics.keys():
        sqlstyle = "SELECT * FROM {table} WHERE g3e_sno = '{styleid}'".format(table = styletable, styleid = styleid)
        df=pd.read_sql(sqlstyle,con=connection)
        data = df.to_dict(orient = 'list')
        # remove useless stuff
        data.pop('G3E_SNO', None)
        data.pop('G3E_USERNAME', None)
        data.pop('G3E_EDITDATE', None)

        styles.update({styleid:data})
else:
    for styleid in logics.keys():
        sqlstyle = "SELECT b.* FROM G3E_COMPOSITELINESTYLE a JOIN g3e_linestyle b on a.g3e_line1 = b.g3e_sno WHERE a.g3e_sno = '{styleid}'".format(styleid = styleid)
        df1=pd.read_sql(sqlstyle,con=connection)
        # remove useless stuff
        df1 = df1.drop(df1.columns[[0,1,3,4,5,6,9,10,11]], axis = 1)

        sqlstyle = "SELECT c.* FROM G3E_COMPOSITELINESTYLE a JOIN g3e_linestyle c on a.g3e_line2 = c.g3e_sno WHERE a.g3e_sno = '{styleid}'".format(styleid = styleid)
        df2=pd.read_sql(sqlstyle,con=connection)
        # remove useless stuff
        df2 = df2.drop(df2.columns[[0,1,3,4,5,6,9,10,11]], axis = 1)

        df = pd.concat([df1,df2], axis = 1)
        # rename column that has the same name
        cols = pd.Series(df.columns)            
        for dup in df.columns[df.columns.duplicated(keep=False)]:
            cols[df.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) if d_idx != 0 else dup for d_idx in range(df.columns.get_loc(dup).sum())])
        df.columns = cols
        data = df.to_dict(orient = 'list')

        if df.empty:
            sqlstyle = "SELECT * FROM G3E_LINESTYLE WHERE g3e_sno = '{styleid}'".format(table = styletable, styleid = styleid)
            df=pd.read_sql(sqlstyle,con=connection)
            data = df.to_dict(orient = 'list')
            # remove useless stuff
            data.pop('G3E_SNO', None)
            data.pop('G3E_USERNAME', None)
            data.pop('G3E_EDITDATE', None)
        

        styles.update({styleid:data})

print(styles)

{'21220150': {'G3E_COLOR': [255], 'G3E_WIDTH': [0.04], 'G3E_OFFSET': [0], 'G3E_STARTSYMBOL': [None], 'G3E_ENDSYMBOL': [None], 'G3E_STROKEPATTERN': [2000010], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [3], 'G3E_TRANSLUCENCY': [None]}, '21220160': {'G3E_COLOR': [255], 'G3E_WIDTH': [0.04], 'G3E_OFFSET': [0], 'G3E_STARTSYMBOL': [None], 'G3E_ENDSYMBOL': [None], 'G3E_STROKEPATTERN': [2000003], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [3], 'G3E_TRANSLUCENCY': [None]}, '21220170': {'G3E_COLOR': [14459090], 'G3E_WIDTH': [0.04], 'G3E_OFFSET': [0], 'G3E_STARTSYMBOL': [None], 'G3E_ENDSYMBOL': [None], 'G3E_STROKEPATTERN': [2000003], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [3], 'G3E_TRANSLUCENCY': [None]}, '21220180': {'G3E_COLOR': [16776960], 'G3E_WIDTH': [0.04], 'G3E_OFFSET': [0], 'G3E_STARTSYMBOL': [None], 'G3E_ENDSYMBOL': [None], 'G3E_STROKEPATTERN': [2000010], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [3], 'G3E_TRANSLUCENCY': [None]}, '21220190': {'G3E_COLOR': [16776960], 'G3E_WIDTH': [0.04]

In [128]:
df1 = pd.DataFrame.from_dict({(i): styles[i]
                                for i in styles.keys()},
                                orient = 'index')
df2 = pd.DataFrame.from_dict({(i): logics[i]
                                for i in logics.keys()},
                                orient = 'index')

df = pd.concat([df1,df2], axis = 1)

In [129]:
filename = view_name + ".csv"
csvdata = df.to_csv()
with open(filename,'w') as f:
    f.write(csvdata)
    

In [11]:
# close connection
if connection:
    cursor.close()
    connection.close()
    print("Connection closed")

Connection closed
